# JS to-do list:
1. read in multiple files... ensure averaging values of each (day, week, hour), not summing them all.
2. graph...

In [83]:
# print_function for compatibility with Python 3
#from __future__ import print_function 
#print('Print function from python 3')

# NumPy for numerical computing
import numpy as np

# Pandas for DataFrames
import pandas as pd

# Matplotlib for visualization
import matplotlib.pyplot as plt

# display plots in the notebook
%matplotlib inline

# Seaborn for easier visualization
#import seaborn as sns

#import calendar
import datetime

In [84]:
df = pd.read_csv('turnstile_180310.txt')

# list of .txt files to be read
txt_docs_list = ['turnstile_180310.txt', 'turnstile_180317.txt','turnstile_180324.txt', 
                 'turnstile_180331.txt', 'turnstile_180407.txt']

# reading in the files
df_list = []
for each in range(4,5):
    df_list.append(pd.read_csv(txt_docs_list[each]))
    
# concatenating the DFs together as 'df'
df = pd.concat(df_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196988 entries, 0 to 196987
Data columns (total 11 columns):
C/A                                                                     196988 non-null object
UNIT                                                                    196988 non-null object
SCP                                                                     196988 non-null object
STATION                                                                 196988 non-null object
LINENAME                                                                196988 non-null object
DIVISION                                                                196988 non-null object
DATE                                                                    196988 non-null object
TIME                                                                    196988 non-null object
DESC                                                                    196988 non-null object
ENTRIES                           

In [85]:
df.columns = df.columns.str.strip()

In [86]:
df['turnstile_id'] = df['C/A'] + '_' + df['UNIT'] + '_' + df['SCP'] + '_' + df['STATION']
df['date'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y')
df['DATE_TIME'] = df['DATE'] + '_' + df['TIME']
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'],format="%m/%d/%Y_%H:%M:%S")
df.drop(['LINENAME', 'DIVISION'], axis=1, inplace=True)
df.sample(10)

,C/A,UNIT,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,turnstile_id,date,DATE_TIME
9214,A050,R088,00-05-03,CORTLANDT ST,04/02/2018,01:00:00,REGULAR,163382,114676,A050_R088_00-05-03_CORTLANDT ST,2018-04-02,2018-04-02 01:00:00
50366,N067,R012,00-00-01,34 ST-PENN STA,04/06/2018,04:00:00,REGULAR,1959437,2117231,N067_R012_00-00-01_34 ST-PENN STA,2018-04-06,2018-04-06 04:00:00
149047,R227A,R131,01-00-02,23 ST,04/02/2018,05:00:00,REGULAR,3919772,2004084,R227A_R131_01-00-02_23 ST,2018-04-02,2018-04-02 05:00:00
31516,H014,R249,00-00-01,GRAHAM AV,04/05/2018,12:00:00,REGULAR,179218,623139,H014_R249_00-00-01_GRAHAM AV,2018-04-05,2018-04-05 12:00:00
179223,R523,R147,00-06-00,61 ST WOODSIDE,04/01/2018,05:00:00,REGULAR,1476107,1874374,R523_R147_00-06-00_61 ST WOODSIDE,2018-04-01,2018-04-01 05:00:00
158724,R254,R181,01-00-02,110 ST,04/05/2018,00:00:00,REGULAR,13144952,4315525,R254_R181_01-00-02_110 ST,2018-04-05,2018-04-05 00:00:00
176381,R510,R090,00-00-00,39 AV,04/01/2018,04:00:00,REGULAR,2271561,696035,R510_R090_00-00-00_39 AV,2018-04-01,2018-04-01 04:00:00
38413,J028,R004,00-00-00,75 ST-ELDERTS,04/03/2018,20:00:00,REGULAR,512942,291979,J028_R004_00-00-00_75 ST-ELDERTS,2018-04-03,2018-04-03 20:00:00
83999,N334B,R341,00-06-02,75 AV,04/06/2018,00:00:00,REGULAR,297374819,942547837,N334B_R341_00-06-02_75 AV,2018-04-06,2018-04-06 00:00:00
181494,R529,R208,00-06-01,103 ST-CORONA,04/01/2018,16:00:00,REGULAR,4042843,7220838,R529_R208_00-06-01_103 ST-CORONA,2018-04-01,2018-04-01 16:00:00


In [87]:
set(df.DESC.values)

{'RECOVR AUD', 'REGULAR'}

In [88]:
# check if DESC column means anything... it doesn't
df_recov = df[df.DESC == 'RECOVR AUD']
df_regul = df[df.DESC == 'REGULAR']
print(df_recov.ENTRIES.describe(), df_regul.ENTRIES.describe())

count    6.490000e+02
mean     1.154969e+08
std      3.883481e+08
min      0.000000e+00
25%      4.330230e+05
50%      1.661326e+06
75%      6.564570e+06
max      2.008106e+09
Name: ENTRIES, dtype: float64 count    1.963390e+05
mean     3.839749e+07
std      2.019122e+08
min      0.000000e+00
25%      5.033890e+05
50%      2.585166e+06
75%      6.863192e+06
max      2.130009e+09
Name: ENTRIES, dtype: float64


## Scrappy build of new df ('df_final') with ENTRIES/EXITS counts

In [89]:
df_temp=df[['turnstile_id','date','DATE_TIME','ENTRIES','EXITS','STATION', 'DATE', 'TIME']]

In [90]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196988 entries, 0 to 196987
Data columns (total 8 columns):
turnstile_id    196988 non-null object
date            196988 non-null datetime64[ns]
DATE_TIME       196988 non-null datetime64[ns]
ENTRIES         196988 non-null int64
EXITS           196988 non-null int64
STATION         196988 non-null object
DATE            196988 non-null object
TIME            196988 non-null object
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 12.0+ MB


In [91]:
df_temp.sample(10)

,turnstile_id,date,DATE_TIME,ENTRIES,EXITS,STATION,DATE,TIME
102307,N546_R204_00-00-02_CHURCH AV,2018-04-05,2018-04-05 04:00:00,4598855,5432812,CHURCH AV,04/05/2018,04:00:00
23943,C025_R215_00-03-02_86 ST,2018-04-01,2018-04-01 20:00:00,17828274,10785439,86 ST,04/01/2018,20:00:00
98925,N528_R257_01-00-02_EAST BROADWAY,2018-04-01,2018-04-01 04:00:00,5779520,1698169,EAST BROADWAY,04/01/2018,04:00:00
356,A002_R051_02-03-06_59 ST,2018-04-03,2018-04-03 08:00:00,8429207,489158,59 ST,04/03/2018,08:00:00
159040,R257_R182_01-03-01_116 ST,2018-04-01,2018-04-01 17:00:00,1903122,1733415,116 ST,04/01/2018,17:00:00
120900,PTH21_R540_01-00-04_PATH WTC 2,2018-04-02,2018-04-02 13:54:15,223960,719888,PATH WTC 2,04/02/2018,13:54:15
118632,PTH18_R549_01-00-01_NEWARK BM BW,2018-04-04,2018-04-04 06:29:31,158382,10788,NEWARK BM BW,04/04/2018,06:29:31
101164,N541_R241_01-05-00_15 ST-PROSPECT,2018-04-03,2018-04-03 16:30:00,458756,0,15 ST-PROSPECT,04/03/2018,16:30:00
109704,N702A_R572_01-00-00_96 ST-2 AVE,2018-04-05,2018-04-05 13:00:00,626881,581792,96 ST-2 AVE,04/05/2018,13:00:00
147709,R221_R170_01-00-01_14 ST-UNION SQ,2018-04-02,2018-04-02 01:00:00,8066693,3055445,14 ST-UNION SQ,04/02/2018,01:00:00


In [92]:
df_temp1=(df_temp
 .sort_values(by=['turnstile_id','DATE_TIME'])
 .groupby(['turnstile_id','date','STATION'])
 .diff())

In [93]:
df_temp1.head(10)

,DATE_TIME,ENTRIES,EXITS
0,NaT,NaN,NaN
1,04:00:00,7.0,3.0
2,04:00:00,0.0,0.0
3,04:00:00,0.0,2.0
4,04:00:00,0.0,0.0
5,04:00:00,0.0,0.0
6,NaT,NaN,NaN
7,04:00:00,0.0,0.0
8,04:00:00,0.0,0.0
9,04:00:00,0.0,0.0


In [94]:
# Here we are rebuilding our new df with ENTRIES/EXITS counts (from .diff()) and all other relevant information
df_temp2=(df_temp
 .sort_values(by=['turnstile_id','DATE_TIME']))

In [95]:
df_temp2.head()

,turnstile_id,date,DATE_TIME,ENTRIES,EXITS,STATION,DATE,TIME
0,A002_R051_02-00-00_59 ST,2018-03-31,2018-03-31 00:00:00,6566463,2224050,59 ST,03/31/2018,00:00:00
1,A002_R051_02-00-00_59 ST,2018-03-31,2018-03-31 04:00:00,6566470,2224053,59 ST,03/31/2018,04:00:00
2,A002_R051_02-00-00_59 ST,2018-03-31,2018-03-31 08:00:00,6566470,2224053,59 ST,03/31/2018,08:00:00
3,A002_R051_02-00-00_59 ST,2018-03-31,2018-03-31 12:00:00,6566470,2224055,59 ST,03/31/2018,12:00:00
4,A002_R051_02-00-00_59 ST,2018-03-31,2018-03-31 16:00:00,6566470,2224055,59 ST,03/31/2018,16:00:00


In [96]:
df_temp1['turnstile_id']=df_temp2['turnstile_id']

In [97]:
df_temp1['STATION']=df_temp2['STATION']

In [98]:
df_temp1['date']=df_temp2['date']
df_temp1['DATE_TIME']=df_temp2['DATE_TIME']

In [99]:
df_temp1.head(10)

,DATE_TIME,ENTRIES,EXITS,turnstile_id,STATION,date
0,2018-03-31 00:00:00,NaN,NaN,A002_R051_02-00-00_59 ST,59 ST,2018-03-31
1,2018-03-31 04:00:00,7.0,3.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-31
2,2018-03-31 08:00:00,0.0,0.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-31
3,2018-03-31 12:00:00,0.0,2.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-31
4,2018-03-31 16:00:00,0.0,0.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-31
5,2018-03-31 20:00:00,0.0,0.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-31
6,2018-04-01 00:00:00,NaN,NaN,A002_R051_02-00-00_59 ST,59 ST,2018-04-01
7,2018-04-01 04:00:00,0.0,0.0,A002_R051_02-00-00_59 ST,59 ST,2018-04-01
8,2018-04-01 08:00:00,0.0,0.0,A002_R051_02-00-00_59 ST,59 ST,2018-04-01
9,2018-04-01 12:00:00,0.0,0.0,A002_R051_02-00-00_59 ST,59 ST,2018-04-01


In [100]:
df_temp1.dropna(axis=0,inplace=True)

In [101]:
# Confirm temp1 and temp2 are equivalent - Good, move on.
print(df_temp1.turnstile_id[156050])
print(df_temp2.turnstile_id[156050])

R246_R177_00-00-04_68ST-HUNTER CO
R246_R177_00-00-04_68ST-HUNTER CO


# 'df_final' with ENTRIES/EXITS counts

In [102]:
df_final=df_temp1[['ENTRIES', 'EXITS','turnstile_id','STATION','date', 'DATE_TIME']]

In [103]:
df_final.head()

,ENTRIES,EXITS,turnstile_id,STATION,date,DATE_TIME
1,7.0,3.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-31,2018-03-31 04:00:00
2,0.0,0.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-31,2018-03-31 08:00:00
3,0.0,2.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-31,2018-03-31 12:00:00
4,0.0,0.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-31,2018-03-31 16:00:00
5,0.0,0.0,A002_R051_02-00-00_59 ST,59 ST,2018-03-31,2018-03-31 20:00:00


## Looks OK, check the details...

In [104]:
# Mean is negative, which is strange. Given the min val, must be a handful of massively negative outliers
'''
print(df_final.ENTRIES.describe())
print(df_final.EXITS.describe())
'''

'\nprint(df_final.ENTRIES.describe())\nprint(df_final.EXITS.describe())\n'

## Hugely negative values...lots of them.

In [105]:
# We know the data is off, so just correct for it in the multi-doc flow...
'''
neg_en_test = df_final[df_final.ENTRIES < 0]
neg_ex_test = df_final[df_final.EXITS < 0]
sum_en = neg_en_test.ENTRIES.sum()
sum_ex = neg_ex_test.EXITS.sum()
print('negative ENTRIES count: '+str(len(neg_en_test.ENTRIES)))
print('negative EXITS count: '+str(len(neg_ex_test.EXITS)))
print('negative ENTRIES SUM: ' + str(sum_en))
print('negative EXITS SUM: ' + str(sum_ex))
'''

"\nneg_en_test = df_final[df_final.ENTRIES < 0]\nneg_ex_test = df_final[df_final.EXITS < 0]\nsum_en = neg_en_test.ENTRIES.sum()\nsum_ex = neg_ex_test.EXITS.sum()\nprint('negative ENTRIES count: '+str(len(neg_en_test.ENTRIES)))\nprint('negative EXITS count: '+str(len(neg_ex_test.EXITS)))\nprint('negative ENTRIES SUM: ' + str(sum_en))\nprint('negative EXITS SUM: ' + str(sum_ex))\n"

In [106]:
# removing the negatives with new DF... 'nn_df_final' (non-negative_df_final)
#df_final['nn_entries'] = [abs(x) for x in df_final.ENTRIES]
#df_final['nn_exits'] = [abs(x) for x in df_final.EXITS]

# build the lists...
nn_entries_list = [abs(x) for x in df_final.ENTRIES]
nn_exits_list = [abs(x) for x in df_final.EXITS]
# throw them in df_final as columns...
df_final['nn_entries'] = nn_entries_list
df_final['nn_exits'] = nn_exits_list
# df_final.describe() # confirms cleaning has been performed


/Users/jas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/jas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [107]:
# remove anything above or below 99% quantile.
en_q99 = df_final['nn_entries'].quantile(0.99)
ex_q99 = df_final['nn_exits'].quantile(0.99)

#df_final_c = df_final[(df_final['nn_entries'] < en_q99) & (df_final['nn_exits'] < ex_q99)]
#df_final_c.drop(['ENTRIES','EXITS'], axis=1)

df_final_c = df_final[(df_final['nn_entries'] < en_q99) & (df_final['nn_exits'] < ex_q99)].drop(['ENTRIES','EXITS'], axis=1)
df_final_c.rename(columns={'nn_entries': 'nn99_entries', 'nn_exits': 'nn99_exits'}, inplace=True)
df_final_c.head()

,turnstile_id,STATION,date,DATE_TIME,nn99_entries,nn99_exits
1,A002_R051_02-00-00_59 ST,59 ST,2018-03-31,2018-03-31 04:00:00,7.0,3.0
2,A002_R051_02-00-00_59 ST,59 ST,2018-03-31,2018-03-31 08:00:00,0.0,0.0
3,A002_R051_02-00-00_59 ST,59 ST,2018-03-31,2018-03-31 12:00:00,0.0,2.0
4,A002_R051_02-00-00_59 ST,59 ST,2018-03-31,2018-03-31 16:00:00,0.0,0.0
5,A002_R051_02-00-00_59 ST,59 ST,2018-03-31,2018-03-31 20:00:00,0.0,0.0


# use 'df_final'

In [108]:
# df_final_c --> negative counts removed, highest 1% of entries/exits discarded as outliers.
print(df_final_c.columns)
print(df_final_c.index)

Index(['turnstile_id', 'STATION', 'date', 'DATE_TIME', 'nn99_entries',
       'nn99_exits'],
      dtype='object')
Int64Index([     1,      2,      3,      4,      5,      7,      8,      9,
                10,     11,
            ...
            196977, 196978, 196979, 196980, 196981, 196983, 196984, 196985,
            196986, 196987],
           dtype='int64', length=160822)


In [115]:
df_final_date = df_final_c[['STATION', 'date', 'turnstile_id', 'nn99_entries', 'nn99_exits']] # cut DATE_TIME out here..
df_final_date['day_of_week'] = df_final_date['date'].dt.weekday_name
df_final_date.sample(20)

,STATION,date,turnstile_id,nn99_entries,nn99_exits,day_of_week
193987,FLATBUSH AV-B.C,2018-04-05,R645_R110_00-05-00_FLATBUSH AV-B.C,0.0,0.0,Thursday
80051,JKSN HT-ROOSVLT,2018-04-01,N324_R018_00-06-02_JKSN HT-ROOSVLT,600.0,137.0,Sunday
16077,AVENUE J,2018-04-05,B021_R228_00-03-00_AVENUE J,15.0,179.0,Thursday
15087,NEWKIRK PLAZA,2018-04-01,B019_R149_00-00-02_NEWKIRK PLAZA,126.0,279.0,Sunday
128249,14 ST,2018-04-03,R128_R105_01-03-02_14 ST,38.0,6.0,Tuesday
33212,HALSEY ST,2018-04-02,H028_R266_00-00-00_HALSEY ST,546.0,116.0,Monday
93844,42 ST-BRYANT PK,2018-03-31,N504_R021_02-06-01_42 ST-BRYANT PK,16.0,49.0,Saturday
32649,DEKALB AV,2018-04-05,H023_R236_00-06-02_DEKALB AV,196.0,90.0,Thursday
43129,145 ST,2018-04-05,N020_R101_00-00-00_145 ST,835.0,474.0,Thursday
34038,SUTTER AV,2018-03-31,H037_R349_00-00-01_SUTTER AV,31.0,20.0,Saturday


# This is where multi-weeks could get tricky...

### Build df to summarize stations by DATE

In [110]:
#df_final_date = df_final_date.set_index('date')
df_final_date.index

Int64Index([     1,      2,      3,      4,      5,      7,      8,      9,
                10,     11,
            ...
            196977, 196978, 196979, 196980, 196981, 196983, 196984, 196985,
            196986, 196987],
           dtype='int64', length=160822)

## Below is final(ish) data for week of April 07, by STATION, by weekday

In [111]:
df_final_date = df_final_date.groupby(['STATION', 'date'])[['nn99_entries', 'nn99_exits']].sum()
df_final_date.reset_index(inplace=True)
df_final_date['day_of_week'] = df_final_date['date'].dt.weekday_name
#df_final_date.sort_values(by=['STATION', 'day_of_week'])
print(df_final_date.head())
print(df_final_date.info()) # 1st week data... 2603 rows. Check with multi-weeks

  STATION       date  nn99_entries  nn99_exits day_of_week
0    1 AV 2018-03-31       12327.0     13248.0    Saturday
1    1 AV 2018-04-01        9766.0     11258.0      Sunday
2    1 AV 2018-04-02       11995.0     12169.0      Monday
3    1 AV 2018-04-03       10583.0     12106.0     Tuesday
4    1 AV 2018-04-04       10791.0     12052.0   Wednesday
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2604 entries, 0 to 2603
Data columns (total 5 columns):
STATION         2604 non-null object
date            2604 non-null datetime64[ns]
nn99_entries    2604 non-null float64
nn99_exits      2604 non-null float64
day_of_week     2604 non-null object
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 101.8+ KB
None


In [ ]:
# to csv for Tableau / visualizations

In [116]:
df_final_date.to_csv('cleanwk_apr07.csv')

In [112]:
# trial 1 to see if resampling of multiple datasets is functional
#df_final_date = df_final_date.set_index('date')
df_final_date.date.resample('D').mean()
df_final_date.groupby('STATION').resample('D').mean() # <<---- this is the key to averaging, not adding, each data set
print(df_final_date.info())
print(df_final_date.head())

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

### Build df to summarize stations by DATE_TIME

In [ ]:
df_final_datetime = df_final_c[['DATE_TIME', 'STATION', 'nn99_entries', 'nn99_exits', 'turnstile_id']]
df_final_datetime = df_final_datetime.groupby(['STATION', 'DATE_TIME'])[['nn99_entries','nn99_exits']].sum()
df_final_datetime.reset_index(inplace=True)
df_final_datetime.sample(10)


## top 10 stations

In [ ]:
top10_stations = df_final_datetime.groupby('STATION')[['nn99_entries','nn99_exits']].sum()
top10_stations.sort_values('nn99_entries', ascending=False, inplace=True)
top10_list = top10_stations.index[:10]
top10_list

## Resampling...

In [ ]:
#df_final_datetime.DATE_TIME.resample('H')
#hourly_counts = df_final_datetime.DATE_TIME.DatetimeIndex.resample('H', how='mean')

In [ ]:
df_final_datetime_dti = df_final_datetime.set_index('DATE_TIME')
df_final_datetime_dti.index

## Good until here --> keep going!

In [ ]:
dti_re = df_final_datetime_dti.resample('H')
dti_re

In [ ]:
df_final_datetime_dti.backfill()

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.resample.Resampler.aggregate.html
''' upsampled = series.resample('D')
    interpolated = upsampled.interpolate(method='spline', order=2)
'''
dti_re.agg({'STATIONS': lambda x: x in x, 'nn99_entries': lambda x: x/4, 'nn99_exits': lambda x: x/4})

## testing resampling processes...
- 1. use lambda function within resampled series to define what exactly to do with other column values
- 2. cut resampled info out to build it, then rematch it to dataframe with merge/join/other???

## Now to visualize...

# - BONUS - We can drill into specific stations for visuals, if wanted / needed...

In [ ]:
df_59st = df_final_datetime[df_final_datetime.STATION == '59 ST']
df_59st.sample(10)

In [ ]:
#df_59st.groupby('day_of_week')[['ENTRIES','EXITS']].sum()

In [ ]:
kelly_df = df_final_datetime.groupby(['STATION', 'date'], as_index=False).sum()
kelly_df.set_index('STATION', inplace=True)
kelly_df.sample(50)

In [ ]:
#kelly_df.to_csv(r'\Users\jas\Desktop\jstationcsv.csv')